In [1]:
# Import the required libraries and dependencies
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.preprocessing import StandardScaler
import hvplot.pandas

In [2]:
# Read the CSV file into a Pandas DataFrame
# Set the index using the Ticker column
rate_df = pd.read_csv(
    Path("../Resources/global_carry_trades.csv"))

# Review the DataFrame
rate_df.head()

,interest_differential,next_month_currency_return,IMF Country Code
0,0.001414,-0.061174,GBR
1,-0.000570,-0.058120,BEL
2,0.001478,-0.056031,DNK
3,0.000655,-0.056991,FRA
4,-0.002928,-0.067056,DEU


## Prepare the Data 

In [3]:
# Use the StandardScaler module and fit_transform function to 
# scale all columns with numerical values
rate_df_scaled = StandardScaler().fit_transform(rate_df[["interest_differential" , "next_month_currency_return"]])

# Diplay the first three rows of the scaled data
rate_df_scaled[0:3]

array([[-0.24270991, -1.93608838],
       [-0.8539933 , -1.84109498],
       [-0.22308154, -1.77613322]])

In [4]:
# Create a DataFrame called with the scaled data
# The column names should match those referenced in the StandardScaler step
rate_df_scaled = pd.DataFrame(
    rate_df_scaled,
    columns=["interest_differential" , "next_month_currency_return"])
rate_df_scaled

,interest_differential,next_month_currency_return
0,-0.242710,-1.936088
1,-0.853993,-1.841095
2,-0.223082,-1.776133
3,-0.476617,-1.805994
4,-1.580459,-2.119073
...,...,...
994,0.122649,-0.846237
995,-0.038476,-0.722418
996,-2.065714,-0.113693
997,-0.283230,-1.169689


In [5]:
# Encode (convert to dummy variables) the "IMF Country Code" column
countries_encoded = pd.get_dummies(rate_df['IMF Country Code'])

# Review the DataFrame
countries_encoded.head()

,AUS,BEL,CAN,CHE,DEU,DNK,FRA,GBR,ITA,JPN,NLD,NOR,NZL,SGP,SWE
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [6]:
# Concatenate the scaled data DataFrame with the "IMF Country Code" encoded dummies 
rate_df_scaled = pd.concat([rate_df_scaled, countries_encoded], axis=1)

# Display the combined DataFrame.
rate_df_scaled.head()

,interest_differential,next_month_currency_return,AUS,BEL,CAN,CHE,DEU,DNK,FRA,GBR,ITA,JPN,NLD,NOR,NZL,SGP,SWE
0,-0.242710,-1.936088,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,-0.853993,-1.841095,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,-0.223082,-1.776133,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,-0.476617,-1.805994,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,-1.580459,-2.119073,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


## Fit and Predict with KMeans


In [7]:
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=3)

# Fit the model for the rate_df_scaled DataFrame
model.fit(rate_df_scaled)

# Save the predicted model clusters to a new DataFrame.
country_clusters = model.predict(rate_df_scaled)

# View the country clusters
print(country_clusters)

C:\Users\arin_\.conda\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\arin_\.conda\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


[1 0 1 1 0 1 0 1 1 0 2 0 1 2 0 2 0 2 0 0 2 0 2 2 0 2 0 2 2 0 2 0 2 2 0 2 0
 2 2 0 2 0 2 1 0 0 0 1 0 0 1 0 2 2 0 2 0 2 2 0 0 0 0 0 0 0 0 1 0 0 2 0 2 2
 0 1 0 1 1 0 1 0 1 1 0 2 0 1 1 0 0 0 0 0 0 0 0 1 0 0 2 0 2 2 0 1 0 0 0 0 1
 0 1 0 0 0 0 2 1 0 2 0 0 0 0 2 0 2 2 0 2 0 1 2 0 2 2 2 2 0 2 0 2 2 0 2 0 2
 2 0 2 0 0 0 0 2 0 2 2 0 2 0 2 2 0 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0
 1 0 1 1 0 2 0 2 1 0 2 2 2 0 2 2 2 2 2 0 0 0 1 2 0 1 0 0 0 0 1 0 0 0 0 2 0
 2 2 0 2 0 0 0 0 2 0 2 2 0 2 0 1 2 0 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0
 0 2 0 0 2 0 2 0 2 1 0 2 2 2 2 0 2 0 2 2 0 2 0 2 2 0 1 0 1 0 0 1 0 1 1 0 2
 0 2 2 0 2 2 2 2 0 2 0 2 2 0 2 0 2 2 0 1 2 2 2 0 2 0 2 2 0 2 0 2 2 0 2 2 2
 2 0 2 2 2 2 0 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 0 2 0 2 2 0 1 0
 1 2 0 2 2 2 0 2 0 2 2 0 2 0 1 1 0 1 2 2 0 2 0 2 2 0 2 0 2 1 0 2 2 2 0 2 0
 2 2 2 2 0 2 2 0 2 0 0 0 1 0 2 2 0 2 0 2 2 2 2 2 2 0 2 0 2 2 0 2 0 2 2 0 2
 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 0 0 0 0 0 2 2 2 0 2 2 2 0 0 0 2 2 0 2
 2 2 2 0 2 2 2 2 2 2 2 2 

In [8]:
# Create a copy of the concatenated DataFrame
rate_scaled_predictions = rate_df_scaled.copy()

# Create a new column in the copy of the concatenated DataFrame with the predicted clusters
rate_scaled_predictions["CountryCluster"] = country_clusters

# Review the DataFrame
rate_scaled_predictions.head()

,interest_differential,next_month_currency_return,AUS,BEL,CAN,CHE,DEU,DNK,FRA,GBR,ITA,JPN,NLD,NOR,NZL,SGP,SWE,CountryCluster
0,-0.242710,-1.936088,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,1
1,-0.853993,-1.841095,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,0
2,-0.223082,-1.776133,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,1
3,-0.476617,-1.805994,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,1
4,-1.580459,-2.119073,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,0


## Plot and Analyze the Results

In [9]:
# Group the saved DataFrame by cluster using `groupby` to calculate average currency returns
rate_scaled_predictions.groupby(by=['CountryCluster'])['next_month_currency_return'].mean()

CountryCluster
0   -0.242211
1   -1.134452
2    0.628660
Name: next_month_currency_return, dtype: float64

In [10]:
rate_scaled_predictions.hvplot.scatter(
    x="interest_differential",
    y="next_month_currency_return",
    by="CountryCluster"
)

:NdOverlay   [CountryCluster]
   :Scatter   [interest_differential]   (next_month_currency_return)

* Based on this plot, which cluster of country appears to provide both the highest interest spread and currency return?

## Bonus

In [11]:
# Initialize a Birch model with n_clusters=5
birch_model = Birch(n_clusters=5)

# Fit the model for the df_bitcoin_scaled DataFrame
birch_model.fit(rate_df_scaled)

# Predict the model segments (clusters)
country_clusters = birch_model.predict(rate_df_scaled)

# View the stock segments
print(country_clusters)

# Create a copy of the concatenated DataFrame
rate_scaled_predictions = rate_df_scaled.copy()

# Create a new column in the copy of the concatenated DataFrame with the predicted clusters
rate_scaled_predictions["CountryCluster"] = country_clusters

# Review the DataFrame
rate_scaled_predictions.head()


[3 4 3 4 4 4 4 3 4 4 3 4 3 2 1 3 1 3 1 1 2 1 3 2 4 3 1 3 2 1 2 1 3 1 1 2 1
 3 2 1 3 1 3 2 1 3 1 3 1 1 2 1 3 2 4 3 1 3 2 1 3 1 1 4 4 4 4 3 4 4 3 1 2 2
 1 3 4 3 4 4 4 4 3 4 4 3 4 3 0 1 3 1 1 4 4 4 4 3 4 4 3 1 3 2 1 3 1 1 1 1 2
 1 3 4 4 3 4 3 0 1 3 1 1 1 1 2 1 3 2 1 3 1 3 3 1 2 1 1 1 1 2 1 3 2 1 3 1 2
 3 1 2 1 1 1 1 2 1 3 2 1 3 1 2 1 1 3 1 1 4 4 4 4 3 4 4 3 4 3 3 1 3 4 3 4 4
 4 4 3 4 4 3 4 2 3 1 3 1 3 1 2 2 1 3 2 3 3 1 0 2 1 3 1 1 4 4 4 4 3 4 4 3 4
 3 3 1 3 1 1 1 1 2 1 3 2 1 3 1 3 3 1 3 4 3 4 4 4 4 3 4 4 3 4 0 3 1 3 1 1 1
 1 2 1 3 2 3 3 1 2 3 1 2 1 1 2 1 2 1 3 2 3 3 1 2 3 1 3 4 3 4 4 4 4 3 4 4 3
 4 2 3 1 2 1 1 1 1 2 1 3 2 3 3 1 2 3 1 3 1 1 1 1 2 1 3 2 3 3 4 2 3 1 3 1 1
 2 1 2 1 3 2 3 3 1 2 3 1 2 1 2 1 2 1 3 2 3 3 1 2 3 1 2 3 1 1 2 1 3 2 1 3 1
 3 1 1 3 3 1 1 2 1 3 2 3 3 1 3 3 1 3 3 1 1 2 1 3 2 3 3 4 3 3 1 3 3 1 1 2 1
 3 2 3 3 1 3 3 1 2 3 1 1 2 1 3 2 3 3 1 3 3 1 2 3 1 1 2 1 3 2 3 3 1 3 3 1 2
 1 2 1 2 1 3 2 2 3 1 2 3 1 2 1 1 1 1 1 1 4 1 1 1 2 1 1 3 3 1 1 4 1 3 2 3 3
 1 3 3 1 2 1 2 1 2 1 3 2 

,interest_differential,next_month_currency_return,AUS,BEL,CAN,CHE,DEU,DNK,FRA,GBR,ITA,JPN,NLD,NOR,NZL,SGP,SWE,CountryCluster
0,-0.242710,-1.936088,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,3
1,-0.853993,-1.841095,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,4
2,-0.223082,-1.776133,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,3
3,-0.476617,-1.805994,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,4
4,-1.580459,-2.119073,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,4


In [12]:
rate_scaled_predictions.hvplot.scatter(
    x="interest_differential",
    y="next_month_currency_return",
    by="CountryCluster"
)

:NdOverlay   [CountryCluster]
   :Scatter   [interest_differential]   (next_month_currency_return)